### Filtro Basado en el Contenido (ContendBased)

### Vamos a realizar una demostracion de sistema de recomendacion basado en contenido para recomendar items

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import ContendBased as CB

In [2]:
data = pd.read_csv("data.csv")
data.head(10)

,URI,name,text
0,<http://dbpedia.org/resource/Gigi_Rigamonti>,Gigi Rigamonti,gigi rigamonti born 8 april 1949 is an italian...
1,<http://dbpedia.org/resource/Steve_Anderson_(m...,Steve Anderson (musician),steve anderson was briefly a member of the ban...
2,<http://dbpedia.org/resource/Tenzin_Delek_Rinp...,Tenzin Delek Rinpoche,lithang tulku tenzin delek rinpoche or tenzing...
3,<http://dbpedia.org/resource/Gene_Alley>,Gene Alley,leonard eugene alley born july 10 1940 in rich...
4,<http://dbpedia.org/resource/Catherine_Heenan>,Catherine Heenan,catherine heenan full name mary catherine eliz...
5,<http://dbpedia.org/resource/Douglas_H._Paal>,Douglas H. Paal,douglas h paal chinese is vice president for s...
6,<http://dbpedia.org/resource/William_G._Daught...,"William G. Daughtridge, Jr.",william gray bill daughtridge jr is a business...
7,<http://dbpedia.org/resource/Uma_Gajapathi_Raju>,Uma Gajapathi Raju,uma gajapathi raju ba was a member of indian p...
8,<http://dbpedia.org/resource/Danny_Ildefonso>,Danny Ildefonso,danilo ildefonso born december 9 1976 in urdan...
9,<http://dbpedia.org/resource/Mike_Rochford>,Mike Rochford,michael joseph rochford born march 14 1963 in ...


In [3]:
data.tail(10)

,URI,name,text
12891,<http://dbpedia.org/resource/David_Mwiraria>,David Mwiraria,david daudi mwiraria born 3 september 1938 was...
12892,<http://dbpedia.org/resource/Alireza_Sagharchi>,Alireza Sagharchi,alireza sagharchi riba frsa born 1959 is a uni...
12893,<http://dbpedia.org/resource/Sandy_Koufax>,Sandy Koufax,sanford sandy koufax kofks born sanford braun ...
12894,<http://dbpedia.org/resource/Manohar_Mahajan>,Manohar Mahajan,manohar mahajan has been a popular announcer o...
12895,<http://dbpedia.org/resource/Bob_Netolicky>,Bob Netolicky,robert bob netolicky born august 2 1942 is a r...
12896,<http://dbpedia.org/resource/Sara_Mohr-Pietsch>,Sara Mohr-Pietsch,sara mohrpietsch born 1980 is a british music ...
12897,<http://dbpedia.org/resource/Alexander_Romanov...,Alexander Romanovsky (pianist),alexander romanovsky born 1984 is a ukrainian ...
12898,<http://dbpedia.org/resource/Loredana_Groza>,Loredana Groza,loredana groza born 10 june 1970 is a romanian...
12899,"<http://dbpedia.org/resource/Peter,_Sue_and_Marc>","Peter, Sue and Marc",peter sue and marc were a swiss music group fr...
12900,<http://dbpedia.org/resource/Corey_Holcomb>,Corey Holcomb,corey holcomb born september 14 1968 is an ame...


In [4]:
al_pacino=data.query('name == "Al Pacino"')
al_pacino

,URI,name,text
185,<http://dbpedia.org/resource/Al_Pacino>,Al Pacino,alfredo james al pacino ptino born april 25 19...


### Explorar los tags y su peso

###  Primero vamos a vertorizar (extración de los tags) la descripción del producto --> CountVectorizer()
###  Calcular la importancia de cada tag en la descripcion del producto --> TfidfTransformer()

In [5]:
vectorizacion = CountVectorizer()
bag_of_words= vectorizacion.fit_transform(data.text)

In [6]:
bag_words_al_pacino = vectorizacion.transform(al_pacino.text)
CB.mostrar_pesos_tags(bag_words_al_pacino,vectorizacion)

/home/jmoc/Escritorio/notebooks/sistemas recomendacion/ContendBased.py:29: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return resultado.sort([descripcion],ascending=[0]).head(numero)


,peso
tag_peso,
the,16
in,12
for,10
and,9
actor,6
of,5
his,5
he,5


### Normalizamnos los tags para queden valores entre 0 y 1

In [7]:

tf_transformer = TfidfTransformer(use_idf=False).fit(bag_of_words)
matrix_al_pacino=tf_transformer.transform(bag_of_words[al_pacino.index.values])
pesos=CB.mostrar_pesos_tags(matrix_al_pacino,vectorizacion,descripcion='tf')
pesos

/home/jmoc/Escritorio/notebooks/sistemas recomendacion/ContendBased.py:29: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return resultado.sort([descripcion],ascending=[0]).head(numero)


,tf
tag_tf,
the,0.505459
in,0.379094
for,0.315912
and,0.284321
actor,0.189547
of,0.157956
his,0.157956
he,0.157956


### Problema
    ### Los tags mas importantes no describen los items que estamos evaluando

### SOLUCION: Normalizar y poderar teniendo en cuenta la frecuencia de cada tag en los demas documentos

In [8]:
#Eliminar las palabras mas comunes --> stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stopWords = stopwords.words("english")
stopWords

tfidf_vectorizer = TfidfVectorizer( stop_words=stopWords, token_pattern='(?u)\\b[a-zA-Z]\\w\\w+\\b')
tfidf_vectorizer.fit(data.text)

[nltk_data] Downloading package stopwords to /home/jmoc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', '...aven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b[a-zA-Z]\\w\\w+\\b', tokenizer=None,
        use_idf=True, vocabulary=None)

In [9]:
CB.mostrar_pesos_tags(tfidf_vectorizer.transform(al_pacino.text),tfidf_vectorizer)

/home/jmoc/Escritorio/notebooks/sistemas recomendacion/ContendBased.py:29: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return resultado.sort([descripcion],ascending=[0]).head(numero)


,peso
tag_peso,
actor,0.286297
godfather,0.267092
corleone,0.238781
pacino,0.216391
oscar,0.207496
nominations,0.204887
supporting,0.183157
film,0.148294


### Prediccion
### Una vez que ya hemos extraidos los tags de los items, buscamos por similitud los mas parecidos

### Empleamos la métrica del coseno para medir la similitud entre items

In [10]:
vecinos            = NearestNeighbors(n_neighbors=10,metric='cosine',algorithm='brute')
datos_por_tags     = tfidf_vectorizer.transform(data.text)
vecinos.fit(datos_por_tags)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=10, p=2, radius=1.0)

In [11]:
#Como se ve a continuacion, nos recomiendan items de actores
al_pacino_vectorizado  = tfidf_vectorizer.transform(al_pacino.text)
distancia,indices       = vecinos.kneighbors(al_pacino_vectorizado)
data.iloc[indices[0],:]

,URI,name,text
185,<http://dbpedia.org/resource/Al_Pacino>,Al Pacino,alfredo james al pacino ptino born april 25 19...
12795,<http://dbpedia.org/resource/Ralph_Fiennes>,Ralph Fiennes,ralph nathaniel twisletonwykehamfiennes ref fa...
10267,<http://dbpedia.org/resource/Ethan_Hawke>,Ethan Hawke,ethan green hawke born november 6 1970 is an a...
3474,<http://dbpedia.org/resource/Daniel_Day-Lewis>,Daniel Day-Lewis,sir daniel michael blake daylewis kt born 29 a...
460,<http://dbpedia.org/resource/Jeff_Daniels>,Jeff Daniels,jeffrey warren jeff daniels born february 19 1...
1302,<http://dbpedia.org/resource/Paul_Popplewell>,Paul Popplewell,paul popplewell was born in 1977 he became a p...
5975,<http://dbpedia.org/resource/Bryan_Cranston>,Bryan Cranston,bryan lee cranston born march 7 1956 is an ame...
12069,<http://dbpedia.org/resource/Aamir_Khan>,Aamir Khan,aamir khan pronounced amr xan born mohammed aa...
7167,<http://dbpedia.org/resource/Colin_Morgan>,Colin Morgan,colin morgan born 1 january 1986 is a northern...
2071,<http://dbpedia.org/resource/Chiwetel_Ejiofor>,Chiwetel Ejiofor,chiwetelu umeadi chiwetel ejiofor obe tuwtl di...


### Empleamos la métrica del distancia euclidea(l2) para medir la similitud entre items

In [12]:
vecinos            = NearestNeighbors(n_neighbors=10,metric='l2',algorithm='brute')
datos_por_tags     = tfidf_vectorizer.transform(data.text)
vecinos.fit(datos_por_tags)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='l2',
         metric_params=None, n_jobs=1, n_neighbors=10, p=2, radius=1.0)

### Como se ve a continuacion, nos recomiendan items de actores

In [13]:
al_pacino_vectorizado  = tfidf_vectorizer.transform(al_pacino.text)
distancia,indices       = vecinos.kneighbors(al_pacino_vectorizado)
data.iloc[indices[0],:]

,URI,name,text
185,<http://dbpedia.org/resource/Al_Pacino>,Al Pacino,alfredo james al pacino ptino born april 25 19...
12795,<http://dbpedia.org/resource/Ralph_Fiennes>,Ralph Fiennes,ralph nathaniel twisletonwykehamfiennes ref fa...
10267,<http://dbpedia.org/resource/Ethan_Hawke>,Ethan Hawke,ethan green hawke born november 6 1970 is an a...
3474,<http://dbpedia.org/resource/Daniel_Day-Lewis>,Daniel Day-Lewis,sir daniel michael blake daylewis kt born 29 a...
460,<http://dbpedia.org/resource/Jeff_Daniels>,Jeff Daniels,jeffrey warren jeff daniels born february 19 1...
1302,<http://dbpedia.org/resource/Paul_Popplewell>,Paul Popplewell,paul popplewell was born in 1977 he became a p...
5975,<http://dbpedia.org/resource/Bryan_Cranston>,Bryan Cranston,bryan lee cranston born march 7 1956 is an ame...
12069,<http://dbpedia.org/resource/Aamir_Khan>,Aamir Khan,aamir khan pronounced amr xan born mohammed aa...
7167,<http://dbpedia.org/resource/Colin_Morgan>,Colin Morgan,colin morgan born 1 january 1986 is a northern...
2071,<http://dbpedia.org/resource/Chiwetel_Ejiofor>,Chiwetel Ejiofor,chiwetelu umeadi chiwetel ejiofor obe tuwtl di...


### Con todo lo anterior, ya estamos preparados para crear un nuestro sistema de recomendacion basado en contenido

### Buscamos los actores que han podido ganar algun oscar

In [14]:
content_based = CB.ContentBased(stop_words=stopWords, token_pattern='(?u)\\b[a-zA-Z]\\w\\w+\\b')
content_based.fit(data,columna_descripcion='text')

In [15]:
content_based.predict(['Award Oscar'])

,URI,name,text
185,<http://dbpedia.org/resource/Al_Pacino>,Al Pacino,alfredo james al pacino ptino born april 25 19...
3098,<http://dbpedia.org/resource/Scott_Mathews>,Scott Mathews,scott mathews born july 25 1955 is an american...
10267,<http://dbpedia.org/resource/Ethan_Hawke>,Ethan Hawke,ethan green hawke born november 6 1970 is an a...
5227,<http://dbpedia.org/resource/Barbra_Streisand>,Barbra Streisand,barbra joan streisand born barbara joan streis...
5920,<http://dbpedia.org/resource/Janet_McTeer>,Janet McTeer,janet mcteer obe born 5 august 1961 is an engl...
107,<http://dbpedia.org/resource/Imelda_Staunton>,Imelda Staunton,imelda mary philomena bernadette staunton obe ...
583,<http://dbpedia.org/resource/Allison_Janney>,Allison Janney,allison brooks janney born november 19 1959 is...
10576,<http://dbpedia.org/resource/Thomas_Lennon_(fi...,Thomas Lennon (filmmaker),thomas f lennon is a documentary filmmakerthom...
239,<http://dbpedia.org/resource/Lenore_Raphael>,Lenore Raphael,lenore raphael born lenore hyams 1 june 1942 i...
4063,<http://dbpedia.org/resource/Ronald_Fedkiw>,Ronald Fedkiw,ronald paul ron fedkiw fdko born february 27 1...


In [16]:
content_based.predict(['sports'])

,URI,name,text
5725,<http://dbpedia.org/resource/Gregg_Murphy>,Gregg Murphy,gregg murphy is a sports journalist and was pa...
9219,<http://dbpedia.org/resource/Terry_Jones_(jour...,Terry Jones (journalist),terry jones nicknamed large or jonesy born jun...
601,<http://dbpedia.org/resource/Terence_Moore>,Terence Moore,terence moore is an american sports journalist...
7890,<http://dbpedia.org/resource/Tony_Barnhart>,Tony Barnhart,tony barnhart was a reporter for the atlanta j...
10680,<http://dbpedia.org/resource/Jody_Vance>,Jody Vance,jody vance born august 23 1967 in vancouver br...
5858,<http://dbpedia.org/resource/Bill_Brown_(broad...,Bill Brown (broadcaster),bill brown born september 20 1947 in sedalia m...
11262,<http://dbpedia.org/resource/Christian_Jantzen>,Christian Jantzen,christian jantzen is a sydneybased sports repo...
12324,<http://dbpedia.org/resource/John_McDonald_(sp...,John McDonald (sports announcer),john mcdonald sports announcer was born in lon...
9063,<http://dbpedia.org/resource/Charlie_Moore_(te...,Charlie Moore (television personality),charlie moore born november 7 1970 in wakefiel...
6831,<http://dbpedia.org/resource/Mike_Roberts_(spo...,Mike Roberts (sportscaster),mike roberts is a former american radio sports...
